# Deep Neural Network on the CIFAR10 Image Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import numpy as np

In [19]:
cifar = keras.datasets.cifar10
(x_train_full, y_train_full), (x_test, y_test) = cifar.load_data()

In [ ]:
print(x_train_full.shape)
print(x_test.shape)
print(y_train_full.shape)
print(y_test.shape)

# Plot the image

In [ ]:
plt.imshow(x_train_full[1])
plt.axis('off')
plt.show()

In [ ]:
y_train_full[1]

In [25]:
x_train = x_train_full[5000:]
y_train = y_train_full[5000:]
x_valid = x_train_full[:5000]
y_valid = y_train_full[:5000]

# Functions for model code reuse

In [63]:
def create_callbacks(checkpoint_filepath, model_logs_directory):
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
    model_checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True)
    run_index = 1 # increment every time you train the model
    run_logdir = os.path.join(os.curdir, model_logs_directory + "/my_cifar10_logs", "run_bn_{:03d}".format(run_index))
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
    callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
    return callbacks

def evauluate_model(model, x_test, y_test):
    results = model.evaluate(x_test, y_test)
    print("test loss, test accuracy:", results)

def build_model(hidden_layers=1, neurons=10, learning_rate=3e03, input_shape=[8], activation="relu", loss_func = "mse", optimizer=None):
    if optimizer is None:
        print("No optimizer given. Using SGD optimizer with 3e-3 learning rate")
        optimizer = keras.optimizers.SGD(learning_rate=3e-3)
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_shape))

    for _ in range(hidden_layers):
        model.add(keras.layers.Dense(neurons, activation=activation))

    model.add(keras.layers.Dense(10))

    
    model.compile(loss=loss_func, optimizer=optimizer)
    return model

# Build the Network

In [79]:
# static, throughout the notebook
layers = 20
neurons = 100

In [70]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(layers):
    model.add(keras.layers.Dense(neurons,
                                activation="elu",
                                kernel_initializer="he_normal"))

What is He Normal Initialization?

A weight initialization used with ReLU and its variants. Helps avoid issues like vanishing or exploding gradrients during training, which leads to faster convergence and better performance.

In [71]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [72]:
# Cross entropy loss function is an optimization function 
# which is used in case of training a classification model 
# which classifies the data by predicting the probability 
# of whether the data belongs to one class or the other class
loss_fn = keras.losses.SparseCategoricalCrossentropy()

In [73]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss=loss_fn, optimizer=optimizer, metrics=["accuracy"])

In [74]:
#output layer
model.add(keras.layers.Dense(10, activation="softmax"))

In [75]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [76]:
checkpoint_filepath = "my_cifar10_model.keras"
model_logs_directory = "my_model_logs"

callbacks = create_callbacks(checkpoint_filepath, model_logs_directory)

In [ ]:
model.fit(x_train, y_train, epochs=100,
          validation_data=(x_valid, y_valid),
          callbacks=callbacks)

In [ ]:
model = keras.models.load_model(checkpoint_filepath)
evauluate_model(model=model, x_test=x_valid, y_test=y_valid)

This model sucks, lets optimize

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())

for _ in range(layers):
    model.add(keras.layers.Dense(neurons, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))

model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

checkpoint_filepath = "my_cifar10_bn_model.keras"
model_logs_directory = "my_model_logs"

callbacks = create_callbacks(checkpoint_filepath=checkpoint_filepath, model_logs_directory=model_logs_directory)

model.fit(x_train, y_train, epochs=100,
          validation_data=(x_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model(checkpoint_filepath)
model.evaluate(x_valid, y_valid)

In [ ]:
evauluate_model(model=model, x_test=x_valid, y_test=y_valid)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(layers):
    model.add(keras.layers.Dense(neurons,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

checkpoint_filepath = "my_cifar10_selu_model.keras"
model_logs_directory = "my_model_logs"

callbacks = create_callbacks(checkpoint_filepath, model_logs_directory)

In [ ]:
X_means = x_train.mean(axis=0)
X_stds = x_train.std(axis=0)
X_train_scaled = (x_train - X_means) / X_stds
X_valid_scaled = (x_valid - X_means) / X_stds
X_test_scaled = (x_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model(checkpoint_filepath)
model.evaluate(X_valid_scaled, y_valid)

In [ ]:
evauluate_model(model=model, x_test=X_valid_scaled, y_test=y_valid)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(layers):
    model.add(keras.layers.Dense(neurons,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.Dropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

checkpoint_filepath = "my_cifar10_alpha_dropout_model.keras"
model_logs_directory = "my_model_logs"

callbacks = create_callbacks(checkpoint_filepath, model_logs_directory)

In [ ]:
X_means = x_train.mean(axis=0)
X_stds = x_train.std(axis=0)
X_train_scaled = (x_train - X_means) / X_stds
X_valid_scaled = (x_valid - X_means) / X_stds
X_test_scaled = (x_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model(checkpoint_filepath)

In [ ]:
evauluate_model(model=model, x_test=X_valid_scaled, y_test=y_valid)

The CIFAR 10 dataset is large, which presents a challenge for beginners such as myself. The challenge is magnified without the use of CNNs.